In [16]:
import google.generativeai as genai
import numpy as np
import pandas as pd

In [8]:
genai.configure(api_key="YOUR_API_KEY")

In [12]:
defaults = {
  'model': 'models/text-bison-001',
  'temperature': 0,
  'candidate_count': 1,
  'top_k': 40,
  'top_p': 0.95,
  'max_output_tokens': 1024,
  'stop_sequences': [],
  'safety_settings': [{"category":"HARM_CATEGORY_DEROGATORY","threshold":4},
                      {"category":"HARM_CATEGORY_TOXICITY","threshold":4},
                      {"category":"HARM_CATEGORY_VIOLENCE","threshold":4},
                      {"category":"HARM_CATEGORY_SEXUAL","threshold":4},
                      {"category":"HARM_CATEGORY_MEDICAL","threshold":4},
                      {"category":"HARM_CATEGORY_DANGEROUS","threshold":4}],
}

In [13]:
text = " "

In [14]:
prompt = f"""Consider the following comment, and carefully answer the questions in each step to conclude whether it is hate speech or not: 
comment: "{text}"
Let's think of it in the following steps:
Q1A: Which of the following identities are mentioned in the text?
'race', 'nationality', 'age', 'political', 'religion', ‘disability’, '(anti-)masker', '(anti-)vaxxer'
Q1B: Are there any individuals mentioned explicitly by their name, answer 'Yes' or 'No'?
Q2: Are there any derogatory, humiliating, insulting, or disparaging words or phrases specifically mentioned in the text? Answer 'Yes' or 'No'. (Note: Colloquially usage of the words should not be considered)
Q3A: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting your selected identities? Answer 'Yes' or 'No'.
Q3B: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting individuals? Answer 'Yes' or 'No'.
Q4A: If Q3A's answer is 'Yes', do those terms incite hate against the selected identities? Answer 'Yes' or 'No'.
Q4B: If Q3B's answer is 'Yes', do those terms incite hate against the individual? Answer 'Yes' or 'No'.
Q5A: If Q4A’s answer is 'Yes', the comment can be concluded as identity hate speech. Tell me your final conclusion: 'Identity Hate' or 'Non-Hate'.
Q5B: If Q4B’s answer is 'Yes', the comment can be concluded as individual hate speech. Tell me your final conclusion: 'Individual Hate' or 'Non-Hate'.
Show your answers in the following format: ['Q1A': , 'Q1B&': , 'Q2': , 'Q3A': , 'Q3B': , 'Q4A': , 'Q4B': , 'Q5A': , 'Q5B': ]
""".format(text=text)

In [ ]:
response = genai.generate_text(
  **defaults,
  prompt=prompt
)

print(response.result)

In [19]:
df = pd.read_csv(' ') # path to the data

In [36]:
def prediction(text):
    text = str(text.lower())
    prompt = f"""Consider the following comment, and carefully answer the questions in each step to conclude whether it is hate speech or not: 
comment: "{text}"
Let's think of it in the following steps:
Q1A: Which of the following identities are mentioned in the text?
'race', 'nationality', 'age', 'political', 'religion', ‘disability’, '(anti-)masker', '(anti-)vaxxer'
Q1B: Are there any individuals mentioned explicitly by their name, answer 'Yes' or 'No'?
Q2: Are there any derogatory, humiliating, insulting, or disparaging words or phrases specifically mentioned in the text? Answer 'Yes' or 'No'. (Note: Colloquially usage of the words should not be considered)
Q3A: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting your selected identities? Answer 'Yes' or 'No'.
Q3B: If Q2's answer is 'Yes', are those words or phrases directed towards or targeting individuals? Answer 'Yes' or 'No'.
Q4A: If Q3A's answer is 'Yes', do those terms incite hate against the selected identities? Answer 'Yes' or 'No'.
Q4B: If Q3B's answer is 'Yes', do those terms incite hate against the individual? Answer 'Yes' or 'No'.
Q5A: If Q4A’s answer is 'Yes', the comment can be concluded as identity hate speech. Tell me your final conclusion: 'Identity Hate' or 'Non-Hate'.
Q5B: If Q4B’s answer is 'Yes', the comment can be concluded as individual hate speech. Tell me your final conclusion: 'Individual Hate' or 'Non-Hate'.
Show your answers in the following format: [Q1A: , Q1B: , Q2: , Q3A: , Q3B: , Q4A: , Q4B: , Q5A: , Q5B: ]
""".format(text=text)

    # print(text)     
    response = genai.generate_text(
        **defaults,
        prompt=prompt
        )
    result = response.result

    return result

In [37]:
# random sample 1000 comments from the dataset
df = df.sample(n=1000, random_state=1)

In [38]:
df.groupby('ground_truth').count()

,index,original_text,text,category,quarter
ground_truth,,,,,
0,608,608,608,608,608
1,392,392,392,392,392


In [40]:
from tqdm import tqdm
tqdm.pandas()

df['palm2_output'] = df['text'].progress_apply(prediction)

100%|██████████| 1000/1000 [19:26<00:00,  1.17s/it]


In [50]:
df.to_csv('../data/results/palm2_output.csv', index=False)

In [47]:
# get the label out
def get_label(x):
    x = str(x).lower()
    # if there are 2 non-hate, then it is non-hate
    if x.count('non-hate') >= 2:
        return 0
    else:
        return 1
    
df['palm2_label'] = df['palm2_output'].apply(get_label)

In [48]:
df.groupby('palm2_label').count()

,index,original_text,text,category,quarter,ground_truth,palm2_output
palm2_label,,,,,,,
0,708,708,708,708,708,708,708
1,292,292,292,292,292,292,289


In [49]:
# check the accuracy, precision, recall, f1 score
tp, fp, tn, fn = 0, 0, 0, 0
for index, row in df.iterrows():
    if row['ground_truth'] == 1:
        if row['palm2_label'] == 1:
            tp += 1
        else:
            fn += 1
    else:
        if row['palm2_label'] == 1:
            fp += 1
        else:
            tn += 1

print('tp: ', tp)
print('fp: ', fp)
print('tn: ', tn)
print('fn: ', fn)

accuracy = (tp + tn) / (tp + fp + tn + fn)
print('accuracy: ｜ precision: ｜ recall: ｜ f1 score: ')
print(accuracy, '｜', tp/(tp+fp), '｜', tp/(tp+fn), '｜', 2*tp/(2*tp+fp+fn))

tp:  260
fp:  32
tn:  576
fn:  132
accuracy: ｜ precision: ｜ recall: ｜ f1 score: 
0.836 ｜ 0.8904109589041096 ｜ 0.6632653061224489 ｜ 0.7602339181286549
